In [1]:
import os
os.getcwd()

'c:\\Users\\12787\\AppData\\Local\\Programs\\Microsoft VS Code'

In [ ]:
import pandas as pd
import openai
from flask import Flask, request, jsonify

# 读取 TSV 文件
def read_table(file_path):
    df = pd.read_csv(file_path, sep='\t')
    return df

# 将表格数据转换为三元组
def convert_table_to_triples(df):
    triples = []
    for _, row in df.iterrows():
        triples.append((row['Entrant'], "has Constructor", row['Constructor']))
        triples.append((row['Constructor'], "uses Chassis", row['Chassis']))
        triples.append((row['Constructor'], "uses Engine", row['Engine']))
        triples.append((row['Entrant'], "has Driver", row['Driver']))
        triples.append((row['Driver'], "has Car No", str(row['No'])))
    return triples

# 使用大语言模型处理三元组
def retrieve_triple_from_table(query, triples):
    triples_description = " | ".join([f"({s}, {p}, {o})" for s, p, o in triples])
    prompt = f"Given the following triples: {triples_description}\nAnswer the query: {query}"
    
    response = openai.Completion.create(
        engine="text-davinci-003",  # 使用适当的语言模型
        prompt=prompt,
        max_tokens=100,
        temperature=0.5
    )
    
    return response.choices[0].text.strip()

# 初始化 Flask 应用
app = Flask(__name__)

# 定义 API 路由
@app.route('/process', methods=['POST'])
def process_query():
    data = request.json
    query = data['query']
    
    # 读取表格并转换为三元组
    df = read_table('/mnt/data/1_table.tsv')  # 读取上传的文件
    triples = convert_table_to_triples(df)
    
    # 使用大语言模型回答查询
    result = retrieve_triple_from
